<a href="https://colab.research.google.com/github/pu-bi/AI-industry-job-experience-for-non-majors/blob/main/2-preprocess/1_BERT_data_transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# text label 분류를 위한 BERT Fine-tuning; 데이터 구축

* 우리는 텍스트가 주어졌을 때, 하나의 레이블을 예측하는 것을 목표로 BERT 모델을 fine-tuning 하는 방법에 대해 다룰 예정입니다.

* 그에 앞서서, 이번 시간엔 일단 데이터를 어떻게 구축해야 하는지에 대해 생각해봅시다.

## 1. 환경 설정 및 데이터 호출

우선 HuggingFace Transformers 와 Datasets 라이브러리를 설치합니다.

* HuggingFace란, 트랜스포머를 기반으로 하는 다양한 모델(transformer.models)과 학습 스크립트(transformer.Trainer)를 구현해 놓은 모듈
* 원래는 PyTorch나 Tensorflow로 layer, model 등을 선언해주고 학습 스크립트도 전부 구현해야 하지만, 허깅 페이스를 사용하면 이런 수고를 덜 수 있음

In [ ]:
!pip install -q transformers datasets
# -q : quiet; 출력을 조금만

이번 실습에서는 여러분이 미리 정제한 데이터를 실습에 사용할 것입니다.

다른 데이터를 사용해보고 싶다면 다음과 같이 하시면 됩니다.

- [huggingface.co](https://huggingface.co/) 에 접속해 "datasets" 탭을 클릭
- "text-classification" 태그를 누르고 사용하고 싶은 데이터셋을 선택

이 [링크](https://huggingface.co/docs/datasets/loading#local-and-remote-files)를 살펴보시면 다양한 방법들이 나와있습니다. 

데이터를 가져오기 위해 드라이브에 마운트합니다.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import load_dataset

# 각자 경로에 맞게 data path를 입력해주세요.
train_data = "/content/drive/MyDrive/joongang/week_2/finetuning/processed_train.csv"
eval_data = "/content/drive/MyDrive/joongang/week_2/finetuning/processed_eval.csv"
test_data = "/content/drive/MyDrive/joongang/week_2/finetuning/processed_test.csv"

dataset = load_dataset('csv', data_files={'train': train_data, 'eval': eval_data, 'test': test_data})

  0%|          | 0/3 [00:00<?, ?it/s]

보시다시피 우리의 데이터셋은 각각 훈련, 검증, 테스트 데이터로 나뉘어 있습니다.


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'utt', '날씨 묻기', '관광지 추천', '숙소 추천', '맛집 추천', '인사', '소개', '기타'],
        num_rows: 20
    })
    eval: Dataset({
        features: ['Unnamed: 0', 'utt', '날씨 묻기', '관광지 추천', '숙소 추천', '맛집 추천', '인사', '소개', '기타'],
        num_rows: 2
    })
    test: Dataset({
        features: ['Unnamed: 0', 'utt', '날씨 묻기', '관광지 추천', '숙소 추천', '맛집 추천', '인사', '소개', '기타'],
        num_rows: 3
    })
})

훈련 데이터 중 첫 번째 데이터를 한번 살펴봅시다.

In [ ]:
example = dataset['train'][0]
example

{'Unnamed: 0': 14,
 'utt': '세상에 이런일이..',
 '날씨 묻기': 0.0,
 '관광지 추천': 0.0,
 '숙소 추천': 0.0,
 '맛집 추천': 0,
 '인사': 0.0,
 '소개': 0.0,
 '기타': 1}

우리가 만든 데이터는 사용자의 발화와 함께 하나의 의도에 대한 정보(=레이블)가 담겨 있음을 확인할 수 있습니다.

이제 레이블들을 담고 있는 리스트를 생성해봅시다.

더불어 레이블들을 정수에 매핑하는 딕셔너리와, 반대로 정수를 레이블들에 매핑하는 딕셔너리도 만들어 봅시다.

In [ ]:
# label 리스트
labels = [label for label in dataset['train'].features.keys() if label not in ['Unnamed: 0', 'utt']]

# 숫자 : label 매핑한 dictionary
id2label = {idx:label for idx, label in enumerate(labels)}

# label : 숫자 매핑한 dictionary
label2id = {label:idx for idx, label in enumerate(labels)}

print("labels :", labels)
print()
print("id2label :", id2label)
print()
print("label2id", label2id)

labels : ['날씨 묻기', '관광지 추천', '숙소 추천', '맛집 추천', '인사', '소개', '기타']

id2label : {0: '날씨 묻기', 1: '관광지 추천', 2: '숙소 추천', 3: '맛집 추천', 4: '인사', 5: '소개', 6: '기타'}

label2id {'날씨 묻기': 0, '관광지 추천': 1, '숙소 추천': 2, '맛집 추천': 3, '인사': 4, '소개': 5, '기타': 6}


## 2. 데이터 전처리

BERT 모델은 입력에 텍스트를 곧바로 넣지는 못합니다.

모델의 입력은 숫자여야 합니다!

그래서 우선 준비된 문자들을 숫자 형태로 만들어 주어야 합니다.

이런 숫자 형태로 변환된 결과를 `input_ids` 라고 부릅니다. 

아래의 코드에선 AutoTokenizer라는 API를 사용할 것인데, 이걸 사용하면 특정 모델이 학습되었을 당시에 사용되었던 토크나이저를 알아서 로드해줍니다.

즉 버트 모델을 훈련할 때 사용된 __사전__을 이용해 문자를 숫자로 바꿔주게 됩니다.


> 보충) 각각의 버트 모델들은 서로 다른 사전을 가지고 있습니다. 따라서 모델을 사용하기 위해서는 해당 모델을 훈련할 때 사용한 사전을 이용해야 합니다.

> 예시)

> korbert 모델의 tokenizer - 1 : 안녕, 2: 하세요 ...

> kobert 모델의 tokenizer - 1: 그래, 2: 안녕 ...





In [ ]:
from transformers import AutoTokenizer
import numpy as np

# 우리가 사용할 버트 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드
model_name = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 훈련된 tokenizer를 이용해 우리가 만든 데이터(텍스트)를 숫자로 변환하는 함수
def preprocess_data(examples):
  text = examples["utt"]
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=20)
  # padding vs truncation
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  labels_matrix = np.zeros((len(text), len(labels)))

  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]
  
  encoding['labels'] = labels_matrix.tolist()

  return encoding

우리는 우리가 불러온 `dataset(텍스트 형태)`를 `preprocess_data` 함수를 이용해 한번에 숫자로 변환(인코딩)할 것입니다.

`preprocess_data` 함수에 대해 자세히 살펴보기에 앞서 `dataset`에 대해 먼저 알아봅시다.

In [ ]:
type(dataset)

datasets.dataset_dict.DatasetDict

이 자료형은 Huggingface의 Datesets 모듈이 제공하는 자료형으로, 다음과 같은 메서드들을 활용할 수 있습니다. [참고](https://woongjoonchoi.github.io/huggingface/Huggingface-Datasets/)

- `dataset.map` : python의 `map` 메서드와 같은 역할을 합니다. 주로, 행(row)별로 전처리가 필요할 때 `map` 메서드를 사용하게 됩니다.
- `dataset.filter` : filter method를 사용하면 특정 조건을 만족하는 dataset dict를 리턴할 수 있습니다.

간단한 예시를 통해 위의 함수가 진행되는 과정을 이해해봅시다.

In [ ]:
examples = dataset["train"][0:2]
examples

{'Unnamed: 0': [14, 13],
 'utt': ['세상에 이런일이..', '나는 배가 고프다'],
 '날씨 묻기': [0.0, 0.0],
 '관광지 추천': [0.0, 0.0],
 '숙소 추천': [0.0, 0.0],
 '맛집 추천': [0, 0],
 '인사': [0.0, 0.0],
 '소개': [0.0, 0.0],
 '기타': [1, 1]}

In [ ]:
text = examples["utt"]
text

['세상에 이런일이..', '나는 배가 고프다']

In [ ]:
# tokenizer를 이용해 숫자로 변환!
encoding = tokenizer(text, padding="max_length", truncation=True, max_length=20)
print(encoding)
print()
print(f"원래 문자 : {text[0]}")
print(f"숫자로 변환된 값 : {encoding['input_ids'][0]}") # input_ids = 숫자
print(f"token_type_ids : {encoding['token_type_ids'][0]}") # 문장 분리
print(f"attention_mask : {encoding['attention_mask'][0]}") # attention 계산할 파트

{'input_ids': [[2, 3991, 2170, 3667, 2210, 2052, 18, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 717, 2259, 1131, 2116, 22779, 2062, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}

원래 문자 : 세상에 이런일이..
숫자로 변환된 값 : [2, 3991, 2170, 3667, 2210, 2052, 18, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
token_type_ids : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
examples.keys()

dict_keys(['Unnamed: 0', 'utt', '날씨 묻기', '관광지 추천', '숙소 추천', '맛집 추천', '인사', '소개', '기타'])

In [ ]:
labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
labels_batch

{'날씨 묻기': [0.0, 0.0],
 '관광지 추천': [0.0, 0.0],
 '숙소 추천': [0.0, 0.0],
 '맛집 추천': [0, 0],
 '인사': [0.0, 0.0],
 '소개': [0.0, 0.0],
 '기타': [1, 1]}

In [ ]:
labels_matrix = np.zeros((len(text), len(labels)))
for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]
    
labels_matrix

array([[0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.]])

In [ ]:
encoding['labels'] = labels_matrix.tolist()
encoding['labels']

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]

이제 데이터셋 전체를 인코딩해봅시다.

앞서 소개했던 map 메서드를 활용합니다.

In [ ]:
dataset['train'].column_names

['Unnamed: 0', 'utt', '날씨 묻기', '관광지 추천', '숙소 추천', '맛집 추천', '인사', '소개', '기타']

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20
    })
    eval: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3
    })
})

전체 데이터셋에 대해 인코딩이 완료되었습니다.

인코딩된 결과를 한번 확인해봅시다.

In [ ]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
print(example)

{'input_ids': [2, 3991, 2170, 3667, 2210, 2052, 18, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]}


input_ids를 통해 정수로 변환된 결과를 확인할 수 있습니다.

또한 decode 메서드를 활용하면 정수를 다시 문자로 변환할 수도 있습니다.

In [ ]:
print(f"tokenizer를 이용해 변환된 숫자 값 : {example['input_ids']}")
print(f"tokenizer를 이용해 숫자를 문자로 되돌린 값 : {tokenizer.decode(example['input_ids'])}")

# tokenizer의 사전 확인해보기
for ids in example['input_ids']:
  print(f"{ids} = {tokenizer.decode(ids)}")

tokenizer를 이용해 변환된 숫자 값 : [2, 3991, 2170, 3667, 2210, 2052, 18, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tokenizer를 이용해 숫자를 문자로 되돌린 값 : [CLS] 세상에 이런일이.. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
2 = [CLS]
3991 = 세상
2170 = ##에
3667 = 이런
2210 = ##일
2052 = ##이
18 = .
18 = .
3 = [SEP]
0 = [PAD]
0 = [PAD]
0 = [PAD]
0 = [PAD]
0 = [PAD]
0 = [PAD]
0 = [PAD]
0 = [PAD]
0 = [PAD]
0 = [PAD]
0 = [PAD]


labels엔 레이블들이 담겨 있습니다.

In [ ]:
example['labels']

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]

위의 리스트에서 1.0 에 해당하는 label 이 무엇인지 확인해봅시다.

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['기타']

최종적으로, 우리의 데이터를 PyTorch tensors의 형식으로 설정합니다.

이 과정을 거치면 우리의 훈련, 검증, 테스트 데이터가 표준 PyTorch 데이터셋이 됩니다.

In [ ]:
encoded_dataset.set_format("torch")